# Making the baseline model and getting things to work

In [28]:
from utils.loading import load_labels, load_images, load_masked_images
from train.train import train
from train.normalize import normalize
import torch
from torch.utils.data import TensorDataset
from models.simple_convnet import SimpleConvNet
from torch.utils.data import DataLoader
import torch.nn as nn

print("Loading data...")
x_train_dict = load_images()
y_train_dict = load_masked_images()

x_train = [torch.tensor(each['image']) for each in x_train_dict.values()]
y_train = [torch.tensor(each['image']) for each in y_train_dict.values()]

Loading data...


In [30]:
x_train_tensor = torch.stack(x_train, dim=0)  # Shape: [num_samples, 12, 1024, 1024]
y_train_tensor = torch.stack(y_train, dim=0).squeeze(1).long()   # Shape: [num_samples, 1, 1024, 1024]

In [31]:
x_train_tensor

tensor([[[[ 272.0000,  268.0000,  268.0000,  ...,  260.5000,  260.5000,
            260.5000],
          [ 233.0000,  216.0000,  216.0000,  ...,  267.0000,  267.0000,
            267.0000],
          [ 233.0000,  216.0000,  216.0000,  ...,  267.0000,  267.0000,
            267.0000],
          ...,
          [ 618.5000,  636.5000,  636.5000,  ...,  621.5000,  621.5000,
            621.5000],
          [ 618.5000, 1097.0000, 1097.0000,  ...,  621.5000,  621.5000,
            621.5000],
          [ 618.5000, 1097.0000, 1097.0000,  ...,  621.5000,  621.5000,
            621.5000]],

         [[ 260.0000,  228.0000,  228.0000,  ...,  201.5000,  231.0000,
            238.5000],
          [ 250.0000,  239.0000,  239.0000,  ...,  224.5000,  228.5000,
            227.5000],
          [ 203.0000,  249.0000,  249.0000,  ...,  227.0000,  209.0000,
            219.0000],
          ...,
          [ 982.5000,  894.0000,  894.0000,  ...,  618.5000,  676.0000,
            655.5000],
          [ 699.00

In [ ]:
x_train_tensor = torch.stack(x_train, dim=0)  # Shape: [num_samples, 12, 1024, 1024]
y_train_tensor = torch.stack(y_train, dim=0).squeeze(1).long()   # Shape: [num_samples, 1, 1024, 1024]

x_train_tensor = torch.nan_to_num(x_train_tensor, nan=0.0)

x_train_tensor = normalize(x_train_tensor)

In [32]:
x_train_tensor = torch.nan_to_num(x_train_tensor, nan=0.0)

x_train_tensor = normalize(x_train_tensor)

In [33]:
x_train_tensor

In [27]:
print(x_train_tensor.shape)

AttributeError: 'NoneType' object has no attribute 'shape'

In [24]:

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
batch_size = 10

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

print("Training model...")
model = SimpleConvNet()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()
n_epochs = 10

losses_train = train(n_epochs, optimizer, model, loss_fn, train_loader)
print(losses_train)

AttributeError: 'NoneType' object has no attribute 'size'

In [2]:
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets, transforms
from torch import nn, optim
import matplotlib.pyplot as plt
import os
import rasterio
import pickle
from dotenv import load_dotenv

load_dotenv()

device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))

print(f"Training on {device}.")

Training on cpu.


### Import data

In [14]:
def import_training_imgs(train_folder):
    x_train= []

    for file in os.listdir(train_folder):
        if file.endswith('.tif'):
            file_path = os.path.join(train_folder, file)
            with rasterio.open(file_path) as src:
                image_np = src.read()  
                image_tensor = torch.from_numpy(image_np).float()
                x_train.append(image_tensor)
    return x_train
x_train = import_training_imgs(os.getenv('IMAGES_PATH'))

In [15]:
x_train 

[tensor([[[ 272.0000,  268.0000,  268.0000,  ...,  260.5000,  260.5000,
            260.5000],
          [ 233.0000,  216.0000,  216.0000,  ...,  267.0000,  267.0000,
            267.0000],
          [ 233.0000,  216.0000,  216.0000,  ...,  267.0000,  267.0000,
            267.0000],
          ...,
          [ 618.5000,  636.5000,  636.5000,  ...,  621.5000,  621.5000,
            621.5000],
          [ 618.5000, 1097.0000, 1097.0000,  ...,  621.5000,  621.5000,
            621.5000],
          [ 618.5000, 1097.0000, 1097.0000,  ...,  621.5000,  621.5000,
            621.5000]],
 
         [[ 260.0000,  228.0000,  228.0000,  ...,  201.5000,  231.0000,
            238.5000],
          [ 250.0000,  239.0000,  239.0000,  ...,  224.5000,  228.5000,
            227.5000],
          [ 203.0000,  249.0000,  249.0000,  ...,  227.0000,  209.0000,
            219.0000],
          ...,
          [ 982.5000,  894.0000,  894.0000,  ...,  618.5000,  676.0000,
            655.5000],
          [ 699.0

In [4]:
len(x_train)

11

In [5]:
def import_training_labels(train_folder):
    y_train = []

    for file in os.listdir(train_folder):
        if file.endswith('.tif'):
            file_path = os.path.join(train_folder, file)
            with rasterio.open(file_path) as src:
                image_np = src.read()
                image_tensor = torch.from_numpy(image_np)
                y_train.append(image_tensor)
    return y_train

In [6]:
y_train = import_training_labels(os.getenv('MASKED_IMAGES_PATH'))

In [7]:
from utils.loading import load_images, load_masked_images

In [17]:
x_train = load_images()

In [18]:
x_train = [torch.tensor(src['image']) for src in x_train.values()]

In [19]:
x_train

[tensor([[[ 272.0000,  268.0000,  268.0000,  ...,  260.5000,  260.5000,
            260.5000],
          [ 233.0000,  216.0000,  216.0000,  ...,  267.0000,  267.0000,
            267.0000],
          [ 233.0000,  216.0000,  216.0000,  ...,  267.0000,  267.0000,
            267.0000],
          ...,
          [ 618.5000,  636.5000,  636.5000,  ...,  621.5000,  621.5000,
            621.5000],
          [ 618.5000, 1097.0000, 1097.0000,  ...,  621.5000,  621.5000,
            621.5000],
          [ 618.5000, 1097.0000, 1097.0000,  ...,  621.5000,  621.5000,
            621.5000]],
 
         [[ 260.0000,  228.0000,  228.0000,  ...,  201.5000,  231.0000,
            238.5000],
          [ 250.0000,  239.0000,  239.0000,  ...,  224.5000,  228.5000,
            227.5000],
          [ 203.0000,  249.0000,  249.0000,  ...,  227.0000,  209.0000,
            219.0000],
          ...,
          [ 982.5000,  894.0000,  894.0000,  ...,  618.5000,  676.0000,
            655.5000],
          [ 699.0

In [20]:

x_train_tensor = torch.stack(x_train, dim=0)  # Shape: [num_samples, 12, 1024, 1024]
y_train_tensor = torch.stack(y_train, dim=0).squeeze(1).long()   # Shape: [num_samples, 1, 1024, 1024]


In [21]:
x_train_tensor = torch.nan_to_num(x_train_tensor, nan=0.0)

In [22]:
x_train_tensor

tensor([[[[ 272.0000,  268.0000,  268.0000,  ...,  260.5000,  260.5000,
            260.5000],
          [ 233.0000,  216.0000,  216.0000,  ...,  267.0000,  267.0000,
            267.0000],
          [ 233.0000,  216.0000,  216.0000,  ...,  267.0000,  267.0000,
            267.0000],
          ...,
          [ 618.5000,  636.5000,  636.5000,  ...,  621.5000,  621.5000,
            621.5000],
          [ 618.5000, 1097.0000, 1097.0000,  ...,  621.5000,  621.5000,
            621.5000],
          [ 618.5000, 1097.0000, 1097.0000,  ...,  621.5000,  621.5000,
            621.5000]],

         [[ 260.0000,  228.0000,  228.0000,  ...,  201.5000,  231.0000,
            238.5000],
          [ 250.0000,  239.0000,  239.0000,  ...,  224.5000,  228.5000,
            227.5000],
          [ 203.0000,  249.0000,  249.0000,  ...,  227.0000,  209.0000,
            219.0000],
          ...,
          [ 982.5000,  894.0000,  894.0000,  ...,  618.5000,  676.0000,
            655.5000],
          [ 699.00

In [ ]:
means = x_train_tensor.mean(dim=(0, 2, 3))
stds = x_train_tensor.std(dim=(0, 2, 3))
print("Means:", means)
print("Stds:", stds)

eps = 1e-7
stds_fixed = stds + eps

normalizer_pipe = transforms.Normalize(means, stds_fixed)

preprocessor = transforms.Compose([
    normalizer_pipe
])

x_train_tensor = [preprocessor(img) for img in x_train_tensor]
x_train_tensor = torch.stack(x_train_tensor, dim=0)


In [ ]:
x_train_tensor.shape

In [ ]:
y_train_tensor.shape

In [ ]:
type(x_train_tensor)

In [26]:
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)


In [ ]:
x_train_tensor

In [28]:
batch_size = 10

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

### Training function

In [35]:
from train.train import train

###  A simple Convolutional Network

In [ ]:
from models.simple_convnet import SimpleConvNet

In [36]:
model = SimpleConvNet()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()


In [ ]:
n_epochs = 10

losses_train = train(n_epochs, optimizer, model, loss_fn, train_loader)
